In [1]:
import os

assert os.environ['CONDA_DEFAULT_ENV']=='ttv'

## TOIs

In [2]:
import sys
sys.path.insert(0, '../code')
from utils import *

In [3]:
df = get_tois(clobber=True)
df.tail()

Saved:  TOIs with TFPWG disposition==FP are removed.


,TIC ID,TOI,Previous CTOI,Master,SG1A,SG1B,SG2,SG3,SG4,SG5,...,Stellar Radius (R_Sun) err,Stellar Metallicity,Stellar Metallicity err,Stellar Mass (M_Sun),Stellar Mass (M_Sun) err,Sectors,Date TOI Alerted (UTC),Date TOI Updated (UTC),Date Modified,Comments
7022,426471754,6871.01,NaN,1,4,1,1,1,4,4,...,0.040000,0.081,0.017,1.000000,0.126188,"43,70,71",2023-12-21,2023-12-21,2023-12-28 12:14:43,L1 candidate
7023,150096001,6872.01,NaN,5,5,5,5,2,5,5,...,0.010000,NaN,NaN,0.456853,0.020234,"43,44,70,71",2023-12-21,2023-12-21,2023-12-28 12:14:43,K2-133 d; marginal TESS QLP detection
7024,67395329,6873.01,NaN,1,4,1,1,1,4,4,...,0.045422,NaN,NaN,0.950000,0.119579,71,2023-12-21,2023-12-21,2023-12-28 12:14:43,period could be half (9.41 d); possible transi...
7025,4426490,6874.01,TIC 4426490.01,3,4,3,3,1,4,4,...,0.081498,-0.014,0.158,1.100000,0.156621,71,2023-12-21,2023-12-21,2023-12-28 12:14:43,multiple stars in pixel
7026,403693870,6875.01,TIC 403693870.01,3,4,3,3,1,4,4,...,NaN,NaN,NaN,1.190000,NaN,71,2023-12-21,2023-12-21,2023-12-28 12:14:43,match to FP/NPC TOI 528 but TFOP SG1 shows thi...


In [4]:
tics = df['TIC ID'].unique()
len(tics)

5790

In [5]:
len(df['TOI'].unique())

6037

In [6]:
nsys = df['TIC ID'].value_counts()
nsys

TIC ID
425997655    5
367630162    5
260647166    4
29781292     4
150428135    4
            ..
130349701    1
281489322    1
130948731    1
387275908    1
403693870    1
Name: count, Length: 5790, dtype: int64

In [7]:
tics_multis = list(nsys[nsys>1].index)
print(len(tics_multis))

195


## TOI 1136 (Dai+2023)
First order MMR: 3:2, 2:1, 3:2, 7:5, and 3:2

In [8]:
col = 'Period (days)'
d = df[df['TIC ID']==142276270].sort_values(by=col)
d[col]

1126     6.258792
1125    12.518728
1128    18.804756
1127    26.317954
Name: Period (days), dtype: float64

In [9]:
# add two outer planets
periods = [4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]
periods

[4.17278, 6.25725, 12.51937, 18.7992, 26.3162, 39.5387]

https://arxiv.org/abs/1208.3312

$$
\frac{p'}{p} \sim \frac{j}{j-1}
$$

proximity to resonance:
$$
\Delta = \frac{p'}{p} \frac{j-1}{j} - 1
$$

In [10]:
get_orbit_pairs(10, order=1)

[(1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8), (8, 9)]

In [11]:
resonances_1st_order, deltas = get_resonant_pairs(periods, order=1, tol=0.01)
resonances_1st_order, deltas

(['3:2 | P=(6.26,4.17) n=(1,2) (delta=0.03%)',
  '2:1 | P=(12.52,6.26) n=(2,3) (delta=0.04%)',
  '3:2 | P=(18.80,12.52) n=(3,4) (delta=0.11%)',
  '3:2 | P=(39.54,26.32) n=(5,6) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.0016314412161837488])

In [12]:
resonances_2nd_order, deltas = get_resonant_pairs(periods, order=2, tol=0.01)
resonances_2nd_order, deltas

(['6:4 | P=(6.26,4.17) n=(1,2) (delta=0.03%)',
  '4:2 | P=(12.52,6.26) n=(2,3) (delta=0.04%)',
  '6:4 | P=(18.80,12.52) n=(3,4) (delta=0.11%)',
  '7:5 | P=(26.32,18.80) n=(4,5) (delta=0.01%)',
  '6:4 | P=(39.54,26.32) n=(5,6) (delta=0.16%)'],
 [0.0003067499364933779,
  0.0003891485876383882,
  0.001072737685682057,
  0.00010182804131475542,
  0.0016314412161837488])

## All TOIs

In [35]:
ttv_1st_order = []       # commensurability in 1 planet pair 
ttv_2nd_order = []
ttv_chain = []           # more than 1 planet pair
deltas1 = []
deltas2 = []
texts = []
for tic in tics_multis:
    dd = df[df['TIC ID']==tic].squeeze()
    periods = dd["Period (days)"].values
    Nplanets = len(periods)
    toi = str(dd.iloc[0].TOI).split('.')[0] 
    
    resonances_1st_order, delta1 = get_resonant_pairs(periods, order=1, tol=0.01);
    resonances_2nd_order, delta2 = get_resonant_pairs(periods, order=2, tol=0.01);
    msg = f"TOI {str(toi).zfill(4)} | TIC {tic} (nplanets={Nplanets}):<br>\n"
    show = False
    if len(resonances_1st_order)>=1:
        msg += f"First Order: {' '.join(resonances_1st_order)}<br>\n"
        ttv_1st_order.append(tic)
        deltas1.append(min(delta1))
        if len(resonances_1st_order)>=2:
            ttv_chain.append(tic)         
        show = True
    if len(resonances_2nd_order)>=1:
        if len(resonances_1st_order)>=1:
            i = int(resonances_1st_order[0].split(':')[0])
            j = int(resonances_2nd_order[0].split(':')[0])
            if j/i!=2:
                msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
                ttv_2nd_order.append(tic)
                deltas2.append(min(delta2))
                show = True
        else:
            msg += f"Second Order: {' '.join(resonances_2nd_order)}\n"
            ttv_2nd_order.append(tic)
            deltas2.append(min(delta2))
            show = True
    if show:
        print(msg)
        texts.append(msg)

TOI 0174 | TIC 425997655 (nplanets=5):<br>
First Order: 2:1 | P=(7.91,3.98) n=(1,2) (delta=0.58%)<br>

TOI 2104 | TIC 367630162 (nplanets=5):<br>
Second Order: 5:3 | P=(3.27,1.98) n=(1,2) (delta=0.85%)

TOI 0282 | TIC 29781292 (nplanets=4):<br>
First Order: 3:2 | P=(84.26,56.00) n=(3,4) (delta=0.30%)<br>

TOI 0700 | TIC 150428135 (nplanets=4):<br>
First Order: 4:3 | P=(37.42,27.81) n=(3,4) (delta=0.93%)<br>

TOI 1136 | TIC 142276270 (nplanets=4):<br>
First Order: 2:1 | P=(12.52,6.26) n=(1,2) (delta=0.01%) 3:2 | P=(18.80,12.52) n=(2,3) (delta=0.14%)<br>

TOI 0270 | TIC 259377017 (nplanets=3):<br>
First Order: 2:1 | P=(11.38,5.66) n=(2,3) (delta=0.52%)<br>

TOI 0663 | TIC 54962195 (nplanets=3):<br>
First Order: 3:2 | P=(7.10,4.70) n=(2,3) (delta=0.77%)<br>

TOI 2269 | TIC 441798995 (nplanets=3):<br>
First Order: 2:1 | P=(2.84,1.42) n=(1,2) (delta=0.00%)<br>

TOI 1730 | TIC 318022259 (nplanets=3):<br>
First Order: 2:1 | P=(12.57,6.23) n=(2,3) (delta=0.98%)<br>

TOI 2267 | TIC 459837008 (n

In [36]:
len(ttv_1st_order), len(ttv_2nd_order)

(22, 8)

In [37]:
import pandas as pd

s1 = pd.DataFrame(np.c_[ttv_1st_order,deltas1], columns=['tic','delta_min'])
s1['1st order'] = True
s1.sort_values(by='delta_min')

,tic,delta_min,1st order
6,441798995.0,0.000023,True
3,142276270.0,0.000091,True
8,459837008.0,0.000158,True
16,384984325.0,0.000488,True
18,455947620.0,0.001917,True
13,143022742.0,0.002051,True
17,303204165.0,0.002446,True
1,29781292.0,0.003004,True
14,153065527.0,0.004089,True
4,259377017.0,0.005161,True


In [38]:
s2 = pd.DataFrame(np.c_[ttv_2nd_order,deltas2], columns=['tic','delta_min'])
s2['2nd order'] = True
s2.sort_values(by='delta_min')

,tic,delta_min,2nd order
4,178155732.0,0.000263,True
5,294471966.0,0.001432,True
3,27491137.0,0.002862,True
2,441739020.0,0.006611,True
1,468979441.0,0.006748,True
7,397362481.0,0.007041,True
6,392681545.0,0.007379,True
0,367630162.0,0.008545,True


In [39]:
res = pd.merge(s1, s2, how='outer')
res = res.replace(np.nan, False)
res['tic'] = res.tic.map(int)
res = res.sort_values(by='delta_min')
res.head()

,tic,delta_min,1st order,2nd order
6,441798995,0.000023,True,False
3,142276270,0.000091,True,False
8,459837008,0.000158,True,False
26,178155732,0.000263,False,True
16,384984325,0.000488,True,False


In [40]:
len(texts), len(res)

(30, 30)

In [41]:
for tic in res.tic:
    idx = [True if i.split()[4]==str(tic) else False for i in texts]
    x = np.array(texts)[idx]
    print(x[0] if len(x)>0 else '')

TOI 2269 | TIC 441798995 (nplanets=3):<br>
First Order: 2:1 | P=(2.84,1.42) n=(1,2) (delta=0.00%)<br>

TOI 1136 | TIC 142276270 (nplanets=4):<br>
First Order: 2:1 | P=(12.52,6.26) n=(1,2) (delta=0.01%) 3:2 | P=(18.80,12.52) n=(2,3) (delta=0.14%)<br>

TOI 2267 | TIC 459837008 (nplanets=3):<br>
First Order: 9:8 | P=(2.29,2.03) n=(1,2) (delta=0.02%)<br>

TOI 0396 | TIC 178155732 (nplanets=3):<br>
Second Order: 5:3 | P=(5.97,3.59) n=(1,2) (delta=0.03%)

TOI 6109 | TIC 384984325 (nplanets=2):<br>
First Order: 3:2 | P=(8.54,5.70) n=(1,2) (delta=0.05%)<br>

TOI 1446 | TIC 294471966 (nplanets=2):<br>
Second Order: 7:5 | P=(6.32,4.51) n=(1,2) (delta=0.14%)

TOI 6022 | TIC 455947620 (nplanets=2):<br>
First Order: 5:4 | P=(1.93,1.55) n=(1,2) (delta=0.19%)<br>

TOI 4323 | TIC 143022742 (nplanets=2):<br>
First Order: 3:2 | P=(21.97,14.68) n=(1,2) (delta=0.21%)<br>

TOI 4647 | TIC 303204165 (nplanets=2):<br>
First Order: 5:4 | P=(11.07,8.83) n=(1,2) (delta=0.24%)<br>

TOI 2076 | TIC 27491137 (nplane

In [42]:
res.to_csv('../data/tics_resonance_from_toi.csv')

In [43]:
np.savetxt('../data/tics_resonance_from_toi.txt', res.tic.tolist(), fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_1st_order.txt', ttv_1st_order, fmt="%d")
np.savetxt('../data/tics_resonance_from_toi_2nd_order.txt', ttv_2nd_order, fmt="%d")

## Transit timing database
* https://ui.adsabs.harvard.edu/abs/2022ApJS..259...62I/abstract

In [44]:
#https://github.com/transit-timing/tt/blob/master/3_database/table4.csv
url = "https://raw.githubusercontent.com/transit-timing/tt/master/3_database/table4.csv"
df2 = pd.read_csv(url)
df2

,System,Orbit number,T_mid,Uncertainty (days),Time System,#,Reference
0,CoRoT-01,-1412,2.454138e+06,0.000470,BJD,1,2009A&A...506..369B
1,CoRoT-01,-1411,2.454140e+06,0.000590,BJD,1,2009A&A...506..369B
2,CoRoT-01,-1410,2.454141e+06,0.000620,BJD,1,2009A&A...506..369B
3,CoRoT-01,-1409,2.454143e+06,0.000390,BJD,1,2009A&A...506..369B
4,CoRoT-01,-1408,2.454144e+06,0.001630,BJD,1,2009A&A...506..369B
...,...,...,...,...,...,...,...
8662,XO-7,214,2.459407e+06,0.000510,BJD_TDB,1,This work
8663,XO-7,215,2.459410e+06,0.000467,BJD_TDB,1,This work
8664,XO-7,216,2.459413e+06,0.000508,BJD_TDB,1,This work
8665,XO-7,217,2.459415e+06,0.000517,BJD_TDB,1,This work
